In [96]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob import TextBlob
from nltk.sentiment.util import mark_negation
import re

# Calculating potisive/negative scores

In [97]:
data = pd.read_csv('train.csv')
data['negative_score'] = 0.0
data['positive_score'] = 0.0

def sentiment_analysis(text):
    pos, neg = [], []
    
    text = text.replace('.', ' .')
    text = text.replace('?', ' ?')
    text = text.replace('!', ' !')
    text = text.replace(',', ' ,')
    text_split= text.split()
    str_neg = mark_negation(text_split, double_neg_flip=True, shallow=False)
    
    for i in range(len(text_split)):
        synsets = wn.synsets(str(text_split[i]))
        if not synsets == []:
            target = synsets[0]
            senti_synset = swn.senti_synset(target.name())
            
            if str_neg[i].find('_NEG') == -1:
                pos.append(senti_synset.pos_score())
                neg.append(senti_synset.neg_score())
            else:
                neg.append(senti_synset.pos_score())
                pos.append(senti_synset.neg_score())

    pos_score = np.mean(pos)
    neg_score = np.mean(neg)
    
    return (pos_score, neg_score)


for index in tqdm(data.index):
    text = data['comment_text'][index]
    pos, neg = sentiment_analysis(text)
    data['positive_score'][index] = pos
    data['negative_score'][index] = neg
    
data.to_csv('train_pos_neg.csv', encoding='utf-8')